 mlflow ui --backend-store-uri sqlite:///mlruns.db

In [32]:
#Importações
import pandas as pd
from sklearn.model_selection import train_test_split
import mlflow
from pycaret.datasets import get_data
from pycaret.classification import *
import os
from io import StringIO
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
import time
from sklearn.model_selection import train_test_split


In [33]:
# importação data set  https://www.kaggle.com/c/kobe-bryant-shot-selection/data
df = pd.read_parquet("../data/raw/dataset_kobe_dev.parquet")
df.shape
# df_origianal.columns

df_prod = pd.read_parquet("../data/raw/dataset_kobe_prod.parquet")

url_docs_artefatos = "../docs/artefatos"

In [34]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'PreparacaoDados'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

In [35]:
#função para adicionar data frame como artefato 
def adicionaDataFrameArtefato(df, titulo,pasta):
    #Salvar o DataFrame como arquivo parquet
    arquivo_path = f"../data/{pasta}/{titulo}.parquet"
    df.to_parquet(arquivo_path, index=False)
    # Registrando o DataFrame resultante como um artefato no MLflow
    mlflow.log_artifact(arquivo_path, f"{titulo}.parquet")

In [36]:
#variavel com caminho do csv temporario, este foi criado para ser salvo como um aterfato no mlflow
csv_path = ''

def dadosNulosArtefato(df,titulo):
    # Contagem de valores nulos por coluna
    valores_nulos_por_coluna = df.isnull().sum()

    # Criando um DataFrame para exibir os resultados
    tabela_valores_nulos = pd.DataFrame(valores_nulos_por_coluna, columns=['Valores Nulos'])

    # Resetando o índice para manter os nomes das colunas
    tabela_valores_nulos.reset_index(inplace=True)

    # Salvando o DataFrame como um arquivo CSV 
    csv_path = f"{url_docs_artefatos}/texto/{titulo}.csv"
    tabela_valores_nulos.to_csv(csv_path, index=False)
    
    #Adicionado um atefato em formato csv com para listar os campos nullos
    mlflow.log_artifact(csv_path, artifact_path= f"{titulo}.csv")
    
   # os.remove(csv_path) 

In [37]:

csv_shape = ""

def shapeArtefato(df, titulo):
    # Obtendo o número de linhas e colunas do DataFrame
    num_linhas = df.shape[0]
    num_colunas = df.shape[1]
    # Criando um DataFrame com as informações
    info_shape = pd.DataFrame({'Número de Linhas': [num_linhas], 'Número de Colunas': [num_colunas]})

    # Salvando o DataFrame como um arquivo CSV 
    csv_shape = f"{url_docs_artefatos}/texto/{titulo}.csv"
    info_shape.to_csv(csv_shape, index=False)
    #Adicionado artefato com a quantidade inicial de linha e colunas 
    mlflow.log_artifact(csv_shape, artifact_path=f"{titulo}.csv")
    #os.remove(csv_shape)

In [38]:
#Criando artefato com os nomes da colunas originais e sua tipagem
def colunasArtefato(df, titulo):
    # Capturando as informações de df.info() em uma string
    info_buffer = StringIO()
    df.info(buf=info_buffer)
    info_text = info_buffer.getvalue()
      # Criando o caminho para a pasta
    
    pasta_path = os.path.join(url_docs_artefatos,'texto')
    os.makedirs(pasta_path, exist_ok=True)
    
    # Salvando o arquivo de texto na pasta
    file_path = os.path.join(pasta_path, f"{titulo}.txt")
    with open(file_path, "w") as file:
        file.write(info_text)
    mlflow.log_text(info_text, artifact_file=f"{titulo}.txt")

In [39]:
def dataFrameArtefato(df, titulo,pasta):
    #Colunas que irão permanecer na base de dados
    colunas_selecionadas = ['lat','lon','minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']
    #excutando a remoção de colunas e linha que tenha valores nuloes    
    df_processado = df[colunas_selecionadas].dropna()
    colunasArtefato(df_processado, f"colunas_{titulo}")
    shapeArtefato(df_processado, f"shape_{titulo}")
    dadosNulosArtefato(df_processado,f"valores_nulos_{titulo}") 
    
    # Registrando o DataFrame resultante como um artefato no MLflow
    adicionaDataFrameArtefato(df_processado,titulo,pasta)
    
 

In [40]:
def boxplot_faixa_dinamica(df,pasta):
   # dfp =  pd.read_parquet("../data/processed/data_filtered.parquet")
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, palette="Set3", linewidth=2.5)
    plt.title("Gráfico de Box Plot - Faixa Dinâmica das Variáveis")
    plt.xlabel("Variáveis")
    plt.ylabel("Valores")
    plt.xticks(rotation=45)
  #  plt.tight_layout()
    
    # Salvar o gráfico como uma imagem temporária
    temp_file =  f"{url_docs_artefatos}/graficos/{pasta}/boxplot_faixa_dinamica.png"
    plt.savefig(temp_file)
    mlflow.log_artifact(temp_file, artifact_path="boxplot_faixa_dinamica.png")
    
  #  plt.show()

In [41]:
def histogramaVariaveis(df,pasta):
    #dfp =  pd.read_parquet("../data/processed/data_filtered.parquet") 
    plt.figure(figsize=(12, 8))
    df.hist(figsize=(12, 8))
    plt.suptitle("Histograma das Variáveis", fontsize=16)
    plt.tight_layout()
    histogram_path =  f"{url_docs_artefatos}/graficos/{pasta}/histograma.png"
    plt.savefig(histogram_path)
    plt.close()
    mlflow.log_artifact(histogram_path, artifact_path="histograma.png")

In [42]:
def matrixCorrelacao(df,pasta):
   # dfp =  pd.read_parquet("../data/processed/data_filtered.parquet")
    plt.figure(figsize=(10, 8))
    sns.heatmap(df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Matriz de Correlação", fontsize=16)
    plt.tight_layout()
    
    # Salvar o gráfico como um arquivo temporário
    temp_file =  f"{url_docs_artefatos}/graficos/{pasta}/matriz_correlacao.png"
    plt.savefig(temp_file)
    plt.close()
    
    mlflow.log_artifact(temp_file, artifact_path="matriz_correlacao.png")


In [43]:
def plotScatterMatrix(df,pasta):
    scatter_matrix(df, diagonal='hist', figsize=(15, 15))
    plt.suptitle("Matriz de Gráficos de Dispersão", y=0.90, fontsize=16)
    plt.tight_layout()
    
    # Salvar o gráfico como um arquivo temporário
    temp_file = f"{url_docs_artefatos}/graficos/{pasta}/scatter_matrix.png"
    plt.savefig(temp_file)
    plt.close()
    mlflow.log_artifact(temp_file, artifact_path="scatter_matrix.png")


In [44]:
def csvDescribe(df, pasta):
      # Obter estatísticas resumidas usando o método describe()
    describe_stats = df.describe()
    
    # Converter as estatísticas resumidas para uma string
    describe_string = describe_stats.to_string()

    # Caminho do arquivo temporário para salvar as estatísticas resumidas
    temp_path = f"{url_docs_artefatos}/{pasta}/describe_statistics.txt"
    
    # Salvar as estatísticas resumidas em um arquivo de texto
    with open(temp_path, "w") as file:
        file.write(describe_string)
    
    # Registrar o arquivo de texto como um artefato no MLflow
    mlflow.log_artifact(temp_path, artifact_path="describe_statistics.txt")


In [45]:
#df.describe()

In [46]:
def faixaDinamica(df):
    # Calcular a faixa dinâmica de cada variável
    range = df.max() - df.min()
    
    # Exportar os dados para um arquivo de texto
    range.to_csv(f'{url_docs_artefatos}/texto/faixa_dinamica.txt', header=False, index=True)

    # Registrar a faixa dinâmica de cada variável
    for column, value in range.items():
        mlflow.log_metric(f"range_{column}", value)

In [47]:
def dividirBaseTreinoTeste(df, train_ratio= 0.8, test_ratio = 0.2):
    # Dividir os dados em conjuntos de treino e teste com estratificação
    # train_data, test_data = train_test_split(df, test_size=0.2, stratify=df['shot_made_flag'])
   # train_ratio=0.8
   # test_ratio = 0.2
    # Dividir os dados em conjuntos de treino e teste com estratificação
    train_data, test_data = train_test_split(df, test_size=1 - train_ratio, stratify=df['shot_made_flag'])

    # Registrar os parâmetros no MLflow
    mlflow.log_param("train_ratio", train_ratio)
    mlflow.log_param("test_ratio", test_ratio)

    # Salvar os conjuntos de treino e teste em arquivos Parquet
    train_data.to_parquet("../data/processed/base_train.parquet", index=False)
    test_data.to_parquet("../data/processed/base_test.parquet", index=False)

    # Registrar as métricas no MLflow
    mlflow.log_metric("base_train_size", len(train_data))
    mlflow.log_metric("base_test_size", len(test_data))

In [48]:
#MLflow
with mlflow.start_run(experiment_id=experiment_id, run_name='PreparaçãoDados'):
   
   colunasArtefato(df,"colunas_iniciais")#Adicionado artefato com os nomes das colunas e sua respecitva tipagem 
   dadosNulosArtefato(df, "valores_nulos") #Adicionado um atefato em formato csv com para listar os campos nullos
   
   shapeArtefato(df,"shape_inicial")#Adicionado artefato com a quatidade de linhas e colunas iniciais
   adicionaDataFrameArtefato(df,"dataset_kobe_dev","raw") #Adiciondo o data frame de desenvolvimento com atefato no mlflow
   adicionaDataFrameArtefato(df_prod,"dataset_kobe_prod","raw") #Adiciondo o data frame de  produção com atefato no mlflow
   dataFrameArtefato(df,"data_filtered","processed") #Processando o data frame removendo algumas colunas e salvando o mesmo no mlflow
   
  
   """
    Os artefatos abaixo são criados após o processamento dos dados. 
    Como esse processo pode levar algum tempo, uma validação foi adicionada para verificar se o arquivo já foi criado.
    Se o arquivo já existe, os artefatos são criados a partir dele. Caso contrário, o processo de criação dos artefatos é iniciado.
   """ 
   file_path = "../data/processed/data_filtered.parquet"
   attempts = 0
   max_attempts=100
   while attempts < max_attempts:
        if os.path.exists(file_path):  # Verifica se o arquivo existe
            data_filtered =  pd.read_parquet("../data/processed/data_filtered.parquet")
            boxplot_faixa_dinamica(data_filtered,"pre_processamento")#adicioando grafico faixa dinamica como artefato
            histogramaVariaveis(data_filtered,"pre_processamento")#adicionado o histograma como artefato
            matrixCorrelacao(data_filtered,"pre_processamento")#adicionando matriz de correlação como artefato
            plotScatterMatrix(data_filtered,"pre_processamento")#adicinando plot scatter marix como artefato 
            csvDescribe(data_filtered, "texto")# # Adiciona o gráfico da faixa dinâmica como artefato
            dividirBaseTreinoTeste(data_filtered)
            faixaDinamica(data_filtered)
            
            
            
            break
        else:
            print(f"O arquivo {file_path} não foi encontrado. Tentando novamente em 5 segundos...")
            time.sleep(5)  # Espera 5 segundos antes de tentar novamente
            attempts += 1
   else:
        print(f"Não foi possível encontrar o arquivo após {max_attempts} tentativas.")


